In [1]:
#load predictions
import os
import h5py
import numpy as np
from meld_classifier.meld_cohort import MeldCohort, MeldSubject
import torch
import pandas as pd


Setting MELD_DATA_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting BASE_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments/co-ripa1/
Setting FS_SUBJECTS_PATH to 


In [2]:
predictions_file_1 = os.path.join('/rds/project/kw350',
             'rds-kw350-meld/experiments',
             'co-ripa1/iteration_21-09-15/ensemble_21-09-15',
             'fold_all/results/',
             'predictions_ensemble_iteration.hdf5')

predictions_file_2 = os.path.join('/rds/project/kw350/rds-kw350-meld/experiments/co-ripa1',
                                       'iteration_21-09-17/ensemble_21-09-20/fold_all/',
                                       'results','predictions_ensemble_iteration.hdf5')

In [3]:
def load_prediction(subject,hdf5):
    results={}
    with h5py.File(hdf5, "r") as f:
        for hemi in ['lh','rh']:
            results[hemi] = f[subject][hemi]['prediction'][:]
    return results

In [4]:
## initialise subjects as being the val data 

subjects = ['MELD_H17_3T_FCD_0092','MELD_H5_3T_C_0005','MELD_H2_15T_C_0002','MELD_H4_15T_FCD_0005',
'MELD_H4_3T_FCD_0008','MELD_H16_3T_FCD_042','MELD_H26_15T_FCD_0006','MELD_H4_3T_FCD_0041','MELD_H17_3T_FCD_0069',
 'MELD_H14_3T_FCD_0028','MELD_H3_3T_C_0029','MELD_H3_3T_C_0025','MELD_H17_3T_FCD_0129','MELD_H21_3T_FCD_0022',
 'MELD_H15_3T_C_0017','MELD_H4_15T_FCD_0022','MELD_H24_3T_FCD_0009','MELD_H12_3T_FCD_0029','MELD_H3_3T_C_0022',
 'MELD_H10_3T_FCD_0006', 'MELD_H17_3T_FCD_0037','MELD_H3_3T_C_0027','MELD_H5_3T_FCD_0016','MELD_H17_3T_FCD_0063','MELD_H5_3T_FCD_0023','MELD_H11_3T_FCD_0026',
 'MELD_H17_3T_FCD_0118','MELD_H2_3T_FCD_0002','MELD_H4_15T_C_0028','MELD_H14_3T_FCD_0006','MELD_H12_3T_FCD_0019','MELD_H14_3T_FCD_0005','MELD_H2_15T_FCD_0002',
 'MELD_H21_3T_FCD_0020','MELD_H24_3T_FCD_0027','MELD_H19_3T_C_010', 'MELD_H10_3T_C_0004','MELD_H3_3T_C_0064',
 'MELD_H17_3T_FCD_0123','MELD_H21_3T_FCD_0045','MELD_H3_3T_C_0002','MELD_H17_3T_FCD_0109',
 'MELD_H15_3T_C_0021','MELD_H11_3T_FCD_0038','MELD_H17_3T_FCD_0059']
 
len(subjects)


45

In [5]:
c = MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat_6.hdf5',
               dataset='MELD_dataset_V6.csv')

            

In [6]:
from meld_graph.training import tp_fp_fn_tn, get_sensitivity, dice_coeff

Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1


In [23]:
#### compute for multiple threshold with function from training

features=['']

df = pd.DataFrame()
values={}
for si,subject in enumerate(subjects):
    print(subject)
    if si%100==0:
        print(si/len(subjects))
        
    results=[]
    try:
        hdf_predictions = os.path.join('/rds/project/kw350/rds-kw350-meld/experiments/co-ripa1',
                                       'iteration_21-09-15/ensemble_21-09-15/fold_all/',
                                       'results','predictions_ensemble_iteration.hdf5')
        result_hemis = load_prediction(subject,hdf_predictions)
        results=result_hemis
    except KeyError:
        hdf_predictions = os.path.join('/rds/project/kw350/rds-kw350-meld/experiments/co-ripa1',
                                       'iteration_21-09-17/ensemble_21-09-20/fold_all/',
                                       'results','predictions_ensemble_iteration.hdf5')
        result_hemis = load_prediction(subject,hdf_predictions)
        results=result_hemis
          
    subj = MeldSubject(subject,cohort=c)
    detected = False
    stats_stored = np.zeros((len(results),4))
    dices = np.zeros((len(results),2))
    values['ID']=subject
    values['group']=subj.group
    pred={}
    target={}
    
    for hemi in ['lh','rh']:
        _, label = subj.load_feature_lesion_data(
            features, hemi=hemi, features_to_ignore=[])
        
        pred[hemi] = results[hemi].astype('bool')
        target[hemi] = label[c.cortex_mask]
    
    preds = torch.as_tensor(np.array(np.hstack([pred['lh'], pred['rh']]))).long()
    targets = torch.as_tensor(np.array(np.hstack([target['lh'], target['rh']]))).long()
    stats_stored=tp_fp_fn_tn(preds, targets)
    dices = dice_coeff(torch.nn.functional.one_hot(preds, num_classes=2),targets)
    values['tp'] = np.array(stats_stored)[0].astype('float')
    values['fp'] = np.array(stats_stored)[1].astype('float')
    values['size pred'] = np.array((preds>0).sum()).astype('float')
    values['dice lesional'] = np.array(dices)[1].astype('float')
    values['dice non-lesional'] = np.array(dices)[0].astype('float')
    df=pd.concat([df, pd.DataFrame([values])])



    

MELD_H17_3T_FCD_0092
0.0
MELD_H5_3T_C_0005
MELD_H2_15T_C_0002
MELD_H4_15T_FCD_0005
MELD_H4_3T_FCD_0008
MELD_H16_3T_FCD_042
MELD_H26_15T_FCD_0006
MELD_H4_3T_FCD_0041
MELD_H17_3T_FCD_0069
MELD_H14_3T_FCD_0028
MELD_H3_3T_C_0029
MELD_H3_3T_C_0025
MELD_H17_3T_FCD_0129
MELD_H21_3T_FCD_0022
MELD_H15_3T_C_0017
MELD_H4_15T_FCD_0022
MELD_H24_3T_FCD_0009
MELD_H12_3T_FCD_0029
MELD_H3_3T_C_0022
MELD_H10_3T_FCD_0006
MELD_H17_3T_FCD_0037
MELD_H3_3T_C_0027
MELD_H5_3T_FCD_0016
MELD_H17_3T_FCD_0063
MELD_H5_3T_FCD_0023
MELD_H11_3T_FCD_0026
MELD_H17_3T_FCD_0118
MELD_H2_3T_FCD_0002
MELD_H4_15T_C_0028
MELD_H14_3T_FCD_0006
MELD_H12_3T_FCD_0019
MELD_H14_3T_FCD_0005
MELD_H2_15T_FCD_0002
MELD_H21_3T_FCD_0020
MELD_H24_3T_FCD_0027
MELD_H19_3T_C_010
MELD_H10_3T_C_0004
MELD_H3_3T_C_0064
MELD_H17_3T_FCD_0123
MELD_H21_3T_FCD_0045
MELD_H3_3T_C_0002
MELD_H17_3T_FCD_0109
MELD_H15_3T_C_0021
MELD_H11_3T_FCD_0038
MELD_H17_3T_FCD_0059


In [24]:
mean_dice_les = df[df['group']=='patient']['dice lesional'].mean()
print(f'mean dice lesional patient : {mean_dice_les}')

mean_dice_les_all = df['dice lesional'].mean()
print(f'mean dice lesional all : {mean_dice_les_all}')

mean_dice_nonles_all = df['dice non-lesional'].mean()
print(f'mean dice non lesional all : {mean_dice_nonles_all}')

sensitivity = len(df[(df['group']=='patient') & (df['tp']>0)])/len(df[(df['group']=='patient')])
print(f'sensitivity patient : {sensitivity}')

specificity = len(df[(df['group']=='control') & (df['fp']==0)])/len(df[(df['group']=='control')])
print(f'specificity controls : {specificity}')

mean dice lesional patient : 0.2306703891372308
mean dice lesional all : 0.31958783227536414
mean dice non lesional all : 0.9953771008385552
sensitivity patient : 0.59375
specificity controls : 0.5384615384615384
